In [107]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import json

import google.generativeai as genai
import typing_extensions as typing

In [108]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [109]:
jokes_path = "/content/drive/MyDrive/LLM project/DATA/shortjokes.csv"
print(jokes_path)

/content/drive/MyDrive/LLM project/DATA/shortjokes.csv


In [110]:
df = pd.read_csv(jokes_path)
print(f"length of the dataset is {len(df)}")

length of the dataset is 231657


In [111]:
df = df.drop_duplicates(subset=['Joke'])
print(f"After exact dedup: {len(df)}")

After exact dedup: 231657


In [112]:
min_len = 65
max_len = 140
df_filtered = df[(df["Joke"].str.len() >= min_len) & (df["Joke"].str.len() <= max_len)]

In [113]:
df_filtered["length"] = df_filtered["Joke"].str.strip().str.len()

/tmp/ipython-input-2038255353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["length"] = df_filtered["Joke"].str.strip().str.len()


In [114]:
bins = [65, 80, 95, 110, 125, 140]
df_filtered["len_bin"] = pd.cut(df_filtered["length"], bins=bins)

/tmp/ipython-input-4075261157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["len_bin"] = pd.cut(df_filtered["length"], bins=bins)


In [115]:
samples_per_bin = 3500 // df_filtered["len_bin"].nunique()

df_sampled = (
    df_filtered.groupby("len_bin", group_keys=False)
      .apply(lambda x: x.sample(n=samples_per_bin, random_state=42))
      .reset_index(drop=True)
)

df_sampled = df_sampled.drop(columns=["length","len_bin"])
df_sampled = df_sampled.sort_values(by="ID", ascending=True)
df_sampled = df_sampled.reset_index(drop=True)
# Save the dataframe to a CSV file
df_sampled.to_csv("/content/drive/MyDrive/LLM project/DATA/sampled_shortjokes.csv", index=False, encoding="utf-8")


/tmp/ipython-input-2653911343.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_filtered.groupby("len_bin", group_keys=False)
/tmp/ipython-input-2653911343.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=samples_per_bin, random_state=42))
